# Creating a Total length of stay model from Discharge Abstract Database

## Creating the one-hot-encoded file

In [25]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
import datetime
import os 


df = pd.read_csv('/home/bell/Documents/work/temp/dad.csv', header = 0)

# Number of characters of ICD and CCI to be included in the model
flag = 3

# Bin the TLOS DV
los_bin=True
treatment = True 


In [26]:
if(flag>0):
    df['D_I10_1'] = df['D_I10_1'].str[:flag]
    df['D_I10_2'] = df['D_I10_2'].str[:flag]
    df['D_I10_3'] = df['D_I10_3'].str[:flag]
    df['D_I10_4'] = df['D_I10_4'].str[:flag]
    df['D_I10_5'] = df['D_I10_5'].str[:flag]

    df['I_CCI_1'] = df['I_CCI_1'].str[:flag]
    df['I_CCI_2'] = df['I_CCI_2'].str[:flag]
    df['I_CCI_3'] = df['I_CCI_3'].str[:flag]
    df['I_CCI_4'] = df['I_CCI_4'].str[:flag]
    df['I_CCI_5'] = df['I_CCI_5'].str[:flag]
    df['I_CCI_6'] = df['I_CCI_6'].str[:flag]
    df['I_CCI_7'] = df['I_CCI_7'].str[:flag]
    df['I_CCI_8'] = df['I_CCI_8'].str[:flag]
    df['I_CCI_9'] = df['I_CCI_9'].str[:flag]
    df['I_CCI_10'] = df['I_CCI_10'].str[:flag]

    

In [27]:
# Take the first five columns of morbidities. DAD has 20.
morbidity = df[['D_I10_1', 'D_I10_2', 'D_I10_3', 'D_I10_4', 'D_I10_5']]

# Take first ten columns of cci interventions. DAD has 25.
interventions = df[['I_CCI_1', 'I_CCI_2','I_CCI_3','I_CCI_4','I_CCI_5', 'I_CCI_6', 'I_CCI_7','I_CCI_8','I_CCI_9','I_CCI_10']]
demographics = df[['SUB_PROV', 'AGRP_F_D', 'GENDER', 'X_FR_I_T', 'ADM_CAT', 'ENT_CODE', 'X_TO_I_T', 'DIS_DISP', 'WGHT_GRP']]
los = df[['TLOS_CAT', 'ACT_LCAT', 'ALC_LCAT']]


In [28]:
if(los_bin):
    los['TLOS_CAT_BIN'] = np.where(los['TLOS_CAT'] >=10, 1, 0)

    # Converts the values to a list
    morbidity['diseases']= morbidity.values.tolist()
    interventions['treatments']=interventions.values.tolist()

    mlb = MultiLabelBinarizer()
    disease_vector = mlb.fit_transform(morbidity['diseases'].dropna())

    # Column names of the dataframe are the class names of multilabel binarizer
    disease_df = pd.DataFrame(data = disease_vector, columns=mlb.classes_)
    treatment_vector = mlb.fit_transform(interventions['treatments'].dropna())
    treatment_df = pd.DataFrame(data = treatment_vector, columns=mlb.classes_)
if(treatment):
    horizontal_stack = pd.concat([demographics, disease_df, treatment_df, los], axis=1)
else:
    horizontal_stack = pd.concat([demographics, disease_df, los], axis=1)

/home/bell/miniconda2/envs/hephaestus-test/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/bell/miniconda2/envs/hephaestus-test/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/bell/miniconda2/envs/hephaestus-test/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [29]:
# Remove the empty string and ZZZ
if(flag>0):
    z = 'Z' * flag
    empty = ' ' * flag
else:
    z = 'ZZZZZZ'
    empty = "  "

horizontal_stack = horizontal_stack.drop(columns=z)
horizontal_stack = horizontal_stack.drop(columns=empty)

In [30]:
# Save the file
file_name = '/home/bell/Documents/work/temp/' + "dad-vector-"+datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")+".csv"
horizontal_stack.to_csv(file_name)
